In [3]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import gaussian
from skimage.segmentation import watershed
from skimage.morphology import disk, dilation, erosion
import os
import glob
import re
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
cadherin_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/flow3_1.4Pa_18h/Cadherins'
nuclei_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3_1.4Pa_18h/Nuclei'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3_1.4Pa_18h/Cell'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all Cadherin .tif files
cadherin_files = glob.glob(os.path.join(cadherin_dir, '*Cadherins*.tif'))
print(f"Found {len(cadherin_files)} Cadherin files to process")

# Process each file
for cadherin_file in cadherin_files:
    # Get filename
    cadherin_filename = os.path.basename(cadherin_file)

    # Extract the common pattern from the filename (everything before _Cadherins)
    match = re.match(r'(.+?)_Cadherins', cadherin_filename)
    if match:
        # Get the prefix pattern - the part before "_Cadherins"
        prefix = match.group(1)

        # Look for corresponding nuclei mask with same prefix
        nuclei_mask_pattern = f"{prefix}_Nuclei*mask.tif"
        nuclei_mask_files = glob.glob(os.path.join(nuclei_dir, nuclei_mask_pattern))

        if nuclei_mask_files:
            nuclei_mask_file = nuclei_mask_files[0]  # Take the first match if multiple files found
        else:
            print(f"WARNING: Nuclei mask not found for {cadherin_filename}, skipping")
            continue
    else:
        print(f"WARNING: Could not parse filename pattern for {cadherin_filename}, skipping")
        continue

    print(f"Processing: {cadherin_filename}")
    print(f"Using nuclei mask: {os.path.basename(nuclei_mask_file)}")

    # Load the images
    cadherin_img = tifffile.imread(cadherin_file)
    nuclei_masks = tifffile.imread(nuclei_mask_file)

    print(f"  Cadherin image shape: {cadherin_img.shape}")
    print(f"  Nuclei mask shape: {nuclei_masks.shape}")

    # Extract Cadherin channel if needed
    if len(cadherin_img.shape) == 2:
        # Single channel image (already Cadherin)
        print("  Detected single-channel Cadherin image")
        membrane_channel = cadherin_img
    elif len(cadherin_img.shape) == 3 and cadherin_img.shape[0] == 3:
        # Format is (C, H, W)
        print("  Detected format: (C, H, W)")
        membrane_channel = cadherin_img[0]  # First channel
    elif len(cadherin_img.shape) == 3 and cadherin_img.shape[2] == 3:
        # Format is (H, W, C)
        print("  Detected format: (H, W, C)")
        membrane_channel = cadherin_img[:, :, 0]  # First channel
    else:
        print(f"  Unexpected image shape: {cadherin_img.shape}. Using first channel/plane.")
        if len(cadherin_img.shape) == 3:
            membrane_channel = cadherin_img[0] if cadherin_img.shape[0] < cadherin_img.shape[1] else cadherin_img[:, :, 0]
        else:
            membrane_channel = cadherin_img

    # Apply Gaussian blur to reduce noise
    print("  Applying Gaussian blur...")
    membrane_smoothed = gaussian(membrane_channel, sigma=1)

    # Calculate morphological gradient
    print("  Calculating morphological gradient...")
    selem = disk(1)  # Adjust radius if needed
    dilated = dilation(membrane_smoothed, selem)
    eroded = erosion(membrane_smoothed, selem)
    membrane_gradient = dilated - eroded

    # Normalize gradient to 0-1 range
    membrane_gradient_norm = (membrane_gradient - membrane_gradient.min()) / (
                membrane_gradient.max() - membrane_gradient.min())

    # Apply watershed segmentation using nuclei as seeds
    print("  Performing watershed segmentation...")
    watershed_output = watershed(membrane_gradient_norm, nuclei_masks, mask=membrane_gradient_norm > 0)

    # Create output filename that preserves the original file's prefix
    output_filename = f"{prefix}_cell_mask.tif"
    cell_mask_path = os.path.join(output_dir, output_filename)
    tifffile.imwrite(cell_mask_path, watershed_output.astype(np.uint32))
    print(f"  Saved cell mask to {cell_mask_path}")

    # Print statistics
    print(f"  Number of nuclei (seeds): {len(np.unique(nuclei_masks)) - 1}")
    print(f"  Number of segmented cells: {len(np.unique(watershed_output)) - 1}")
    print(f"  Processing complete for {cadherin_filename}\n")

print("All processing complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 8 Cadherin files to process
Processing: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Cadherins_focus_bg_tophat.tif
Using nuclei mask: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Nuclei_contrast_bg_tophat_mask.tif
  Cadherin image shape: (1024, 1024)
  Nuclei mask shape: (1024, 1024)
  Detected single-channel Cadherin image
  Applying Gaussian blur...
  Calculating morphological gradient...
  Performing watershed segmentation...
  Saved cell mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3_1.4Pa_18h/Cell/denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_cell_mask.tif
  Number of nuclei (seeds): 366
  Number of segmented cells: 366
  Processing complete for denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Cadherins_focus_bg_tophat.tif

Processing: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq002_Cadherins_focus_bg_tophat.tif
Using nuclei m